In [1]:
!pip install requests beautifulsoup4 pandas numpy tqdm holidays python-dateutil lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s  0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

START_YEAR = 2020
END_YEAR = 2025


In [3]:
df_locations = pd.read_csv("keyword_mapping.csv")
df_locations.head()

,row_index,original_name,normalized_name,province,removed_prefix
0,1,Di tích lịch sử - văn hóa căn cứ địa cách mạng...,căn cứ địa cách mạng Hải Chi,Quảng Ninh,Di tích lịch sử - văn hóa
1,2,Chợ Trung tâm Ba Chẽ,Chợ Trung tâm Ba Chẽ,Quảng Ninh,NaN
2,3,Di tích lịch sử Miếu Ông – Miếu Bà,Miếu Ông – Miếu Bà,Quảng Ninh,Di tích lịch sử
3,4,Phố đi bộ Tiên Yên,Phố đi bộ Tiên Yên,Quảng Ninh,NaN
4,5,"Trung tâm Văn hóa, Thể thao các dân tộc vùng Đ...","Văn hóa, Thể thao các dân tộc vùng Đông Bắc",Quảng Ninh,Trung tâm


In [4]:
# Tạo danh sách tỉnh từ cột 'province' trong file địa điểm
PROVINCES = sorted(df_locations["province"].dropna().unique().tolist())
print("Danh sách tỉnh có trong file:", PROVINCES)

Danh sách tỉnh có trong file: ['An Giang', 'Bà Rịa - Vũng Tàu', 'Bình Dương', 'Bình Phước', 'Bình Thuận', 'Bạc Liêu', 'Bắc Kạn', 'Bắc Ninh', 'Bến Tre', 'Cao Bằng', 'Cà Mau', 'Gia Lai', 'Hoà Bình', 'Huyện An Phú', 'Huyện Bảo Yên', 'Huyện Bắc Hà', 'Huyện Cao Lãnh', 'Huyện Châu Thành', 'Huyện Chư Prông', 'Huyện Chư Păh', 'Huyện Chư Sê', 'Huyện Gia Bình', 'Huyện Gio Linh', 'Huyện Gò Dầu', 'Huyện Hòa Thành', 'Huyện Hướng Hóa', 'Huyện Hải Lăng', 'Huyện Ia Grai', 'Huyện KBang', 'Huyện Lấp Vò', 'Huyện Mường Khương', 'Huyện Quế Võ', 'Huyện Tam Nông', 'Huyện Thoại Sơn', 'Huyện Thuận Thành', 'Huyện Tháp Mười', 'Huyện Tiên Du', 'Huyện Tri Tôn', 'Huyện Triệu Phong', 'Huyện Trảng Bàng', 'Huyện Tân Biên', 'Huyện Tịnh Biên', 'Huyện Vĩnh Linh', 'Huyện Đa Krông', 'Hà Giang', 'Hà Nam', 'Hà Tĩnh', 'Hưng Yên', 'Hải Dương', 'Hậu Giang', 'Khánh Hoà', 'Kon Tum', 'Lai Châu', 'Lào Cai', 'Lâm Đồng', 'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận', 'Phú Thọ', 'Phú Yên', 'Quảng Nam', 'Quảng Ngãi', 'Qu

In [5]:
def create_base_calendar(start_year, end_year):
    all_dates = pd.date_range(
        start=date(start_year, 1, 1),
        end=date(end_year, 12, 31),
        freq="D"
    )
    df = pd.DataFrame({"date": all_dates})
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day_of_week"] = df["date"].dt.dayofweek  # 0 = Monday
    return df

df_calendar = create_base_calendar(START_YEAR, END_YEAR)
df_calendar.tail()

,date,year,month,day_of_week
2187,2025-12-27,2025,12,5
2188,2025-12-28,2025,12,6
2189,2025-12-29,2025,12,0
2190,2025-12-30,2025,12,1
2191,2025-12-31,2025,12,2


In [7]:
from datetime import date
import pandas as pd

holiday_records = []

# Hàm thêm ngày lễ
def add_holiday(d, name, flag, htype, period):
    holiday_records.append({
        "date": d,
        "holiday_name": name,
        "holiday_flag": flag,
        "holiday_type": htype,
        "holiday_period_id": period
    })

# Tết Dương lịch, Valentine, Noel
for y in range(START_YEAR, END_YEAR + 1):
    add_holiday(date(y, 1, 1),
                "Tết Dương lịch",
                1,
                "official",
                "NEWYEAR_" + str(y))

    add_holiday(date(y, 2, 14),
                "Valentine",
                0,
                "cultural",
                "VALENTINE_" + str(y))

    add_holiday(date(y, 12, 24),
                "Noel",
                0,
                "cultural",
                "NOEL_" + str(y))

    add_holiday(date(y, 12, 25),
                "Noel",
                0,
                "cultural",
                "NOEL_" + str(y))

# Giỗ Tổ Hùng Vương (10/3 âm lịch)
gio_to = {
    2020: date(2020, 4, 2),
    2021: date(2021, 4, 21),
    2022: date(2022, 4, 10),
    2023: date(2023, 4, 29),
    2024: date(2024, 4, 18),
    2025: date(2025, 4, 7),
}

for y, d in gio_to.items():
    add_holiday(d,
                "Giỗ Tổ Hùng Vương",
                1,
                "official",
                "GTHV_" + str(y))

# 30/4 – 1/5
for y in range(START_YEAR, END_YEAR + 1):

    add_holiday(date(y, 4, 30),
                "30/4 – 1/5",
                1,
                "official",
                "304_105_" + str(y))

    add_holiday(date(y, 5, 1),
                "30/4 – 1/5",
                1,
                "official",
                "304_105_" + str(y))

# Quốc khánh
for y in range(START_YEAR, END_YEAR + 1):

    add_holiday(date(y, 9, 2),
                "Quốc khánh",
                1,
                "official",
                "QUOCKHANH_" + str(y))

# Trung Thu
trung_thu = {
    2020: date(2020, 10, 1),
    2021: date(2021, 9, 21),
    2022: date(2022, 9, 10),
    2023: date(2023, 9, 29),
    2024: date(2024, 9, 17),
    2025: date(2025, 10, 6),
}

for y, d in trung_thu.items():
    add_holiday(d,
                "Trung Thu",
                0,
                "cultural",
                "TRUNGTHU_" + str(y))

# Tết Âm lịch 2020–2025
tet_mapping = {
    2020: {
        "29_Tet": date(2020, 1, 23),
        "30_Tet": date(2020, 1, 24),
        "M1": date(2020, 1, 25),
        "M2": date(2020, 1, 26),
        "M3": date(2020, 1, 27),
        "M4": date(2020, 1, 28),
        "M5": date(2020, 1, 29),
    },
    2021: {
        "29_Tet": date(2021, 2, 10),
        "30_Tet": date(2021, 2, 11),
        "M1": date(2021, 2, 12),
        "M2": date(2021, 2, 13),
        "M3": date(2021, 2, 14),
        "M4": date(2021, 2, 15),
        "M5": date(2021, 2, 16),
    },
    2022: {
        "29_Tet": date(2022, 1, 29),
        "30_Tet": date(2022, 1, 30),
        "M1": date(2022, 2, 1),
        "M2": date(2022, 2, 2),
        "M3": date(2022, 2, 3),
        "M4": date(2022, 2, 4),
        "M5": date(2022, 2, 5),
    },
    2023: {
        "29_Tet": date(2023, 1, 20),
        "30_Tet": date(2023, 1, 21),
        "M1": date(2023, 1, 22),
        "M2": date(2023, 1, 23),
        "M3": date(2023, 1, 24),
        "M4": date(2023, 1, 25),
        "M5": date(2023, 1, 26),
    },
    2024: {
        "29_Tet": date(2024, 2, 8),
        "30_Tet": date(2024, 2, 9),
        "M1": date(2024, 2, 10),
        "M2": date(2024, 2, 11),
        "M3": date(2024, 2, 12),
        "M4": date(2024, 2, 13),
        "M5": date(2024, 2, 14),
    },
    2025: {
        "29_Tet": date(2025, 1, 28),
        "30_Tet": date(2025, 1, 29),
        "M1": date(2025, 1, 30),
        "M2": date(2025, 1, 31),
        "M3": date(2025, 2, 1),
        "M4": date(2025, 2, 2),
        "M5": date(2025, 2, 3),
    }
}

for y, mapping in tet_mapping.items():
    for key, d in mapping.items():

        if key in ["M1", "M2", "M3"]:
            add_holiday(d,
                        "Tết Âm lịch – " + key,
                        1,
                        "official",
                        "TET_" + str(y))
        else:
            add_holiday(d,
                        "Tết Âm lịch – " + key,
                        0,
                        "tet_extra",
                        "TET_" + str(y))

# Tạo bảng ngày lễ
df_holidays = pd.DataFrame(holiday_records).sort_values("date").reset_index(drop=True)

# Gộp ngày lễ trùng nhau
official_keywords = [
    "Tết Âm lịch",
    "Tết Dương lịch",
    "30/4",
    "1/5",
    "Quốc khánh",
    "Giỗ Tổ"
]

def is_official(name):
    for k in official_keywords:
        if k in name:
            return True
    return False

merged_rows = []

for date_value, group in df_holidays.groupby("date"):

    names = group["holiday_name"].tolist()
    flags = group["holiday_flag"].tolist()
    types = group["holiday_type"].tolist()
    periods = group["holiday_period_id"].tolist()

    if len(names) == 1:
        merged_rows.append({
            "date": date_value,
            "holiday_name": names[0],
            "holiday_flag": flags[0],
            "holiday_type": types[0],
            "holiday_period_id": periods[0]
        })
        continue

    official_list = []
    unofficial_list = []

    for n in names:
        if is_official(n):
            official_list.append(n)
        else:
            unofficial_list.append(n)

    if len(official_list) > 0:

        main_name = official_list[0]

        if len(unofficial_list) > 0:
            note = ", ".join(unofficial_list)
            main_name = main_name + " (" + note + ")"

        merged_rows.append({
            "date": date_value,
            "holiday_name": main_name,
            "holiday_flag": 1,
            "holiday_type": "official",
            "holiday_period_id": periods[names.index(official_list[0])]
        })

    else:

        combined_name = " + ".join(names)

        merged_rows.append({
            "date": date_value,
            "holiday_name": combined_name,
            "holiday_flag": 0,
            "holiday_type": "cultural",
            "holiday_period_id": "MULTI"
        })


df_holidays_final = pd.DataFrame(merged_rows).sort_values("date").reset_index(drop=True)
df_holidays_final["date"] = pd.to_datetime(df_holidays_final["date"])

# Tạo bảng lịch ngày đầy đủ
calendar_records = []

for y in range(START_YEAR, END_YEAR + 1):
    for m in range(1, 13):
        for d in range(1, 32):
            try:
                current_date = date(y, m, d)
                calendar_records.append({"date": current_date})
            except:
                continue

df_calendar_expanded = pd.DataFrame(calendar_records)
df_calendar_expanded["date"] = pd.to_datetime(df_calendar_expanded["date"])


# Merge lịch ngày với bảng ngày lễ
df_merge = df_calendar_expanded.merge(
    df_holidays_final,
    on="date",
    how="left"
)

df_merge["holiday_name"] = df_merge["holiday_name"].fillna("Ngày thường")
df_merge["holiday_flag"] = df_merge["holiday_flag"].fillna(0).astype(int)
df_merge["holiday_type"] = df_merge["holiday_type"].fillna("none")
df_merge["holiday_period_id"] = df_merge["holiday_period_id"].fillna("NONE")

df_calendar_full = df_merge.sort_values("date").reset_index(drop=True)
df_calendar_full.tail(50)

,date,holiday_name,holiday_flag,holiday_type,holiday_period_id
2142,2025-11-12,Ngày thường,0,none,NONE
2143,2025-11-13,Ngày thường,0,none,NONE
2144,2025-11-14,Ngày thường,0,none,NONE
2145,2025-11-15,Ngày thường,0,none,NONE
2146,2025-11-16,Ngày thường,0,none,NONE
2147,2025-11-17,Ngày thường,0,none,NONE
2148,2025-11-18,Ngày thường,0,none,NONE
2149,2025-11-19,Ngày thường,0,none,NONE
2150,2025-11-20,Ngày thường,0,none,NONE
2151,2025-11-21,Ngày thường,0,none,NONE


In [8]:
df = df_calendar_full.copy()

# Tách thông tin thời gian
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday          # 0 = Thứ Hai, 6 = Chủ Nhật
df["weekofyear"] = df["date"].dt.isocalendar().week
df["quarter"] = df["date"].dt.quarter         # 1, 2, 3, 4

# Cờ cuối tuần
df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)

# Gắn mùa theo quý
def get_season_from_quarter(q):
    if q == 1:
        return "Xuân"
    if q == 2:
        return "Hạ"
    if q == 3:
        return "Thu"
    return "Đông"

df["season"] = df["quarter"].apply(get_season_from_quarter)

df_calendar_enriched = df.sort_values("date").reset_index(drop=True)
df_calendar_enriched.head(95)

,date,holiday_name,holiday_flag,holiday_type,holiday_period_id,year,month,day,weekday,weekofyear,quarter,is_weekend,season
0,2020-01-01,Tết Dương lịch,1,official,NEWYEAR_2020,2020,1,1,2,1,1,0,Xuân
1,2020-01-02,Ngày thường,0,none,NONE,2020,1,2,3,1,1,0,Xuân
2,2020-01-03,Ngày thường,0,none,NONE,2020,1,3,4,1,1,0,Xuân
3,2020-01-04,Ngày thường,0,none,NONE,2020,1,4,5,1,1,1,Xuân
4,2020-01-05,Ngày thường,0,none,NONE,2020,1,5,6,1,1,1,Xuân
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2020-03-31,Ngày thường,0,none,NONE,2020,3,31,1,14,1,0,Xuân
91,2020-04-01,Ngày thường,0,none,NONE,2020,4,1,2,14,2,0,Hạ
92,2020-04-02,Giỗ Tổ Hùng Vương,1,official,GTHV_2020,2020,4,2,3,14,2,0,Hạ
93,2020-04-03,Ngày thường,0,none,NONE,2020,4,3,4,14,2,0,Hạ


In [9]:
!pip install requests beautifulsoup4 lxml pandas


In [10]:
import pandas as pd
from itertools import product

# Ví dụ: đọc danh sách tỉnh/thành từ file keyword_mapping.csv
# Giả sử file có cột "province"
df = pd.read_csv("keyword_mapping.csv")
provinces = df["province"].dropna().unique().tolist()

# Sinh tất cả tuyến đường (cartesian product)
routes = []
for origin, dest in product(provinces, provinces):
    if origin != dest:  # bỏ trường hợp đi và đến cùng 1 nơi
        routes.append({"origin": origin, "destination": dest})

df_routes = pd.DataFrame(routes)
print(df_routes.head())
print("Tổng số tuyến:", len(df_routes))

       origin           destination
0  Quảng Ninh           Tuyên Quang
1  Quảng Ninh      Thành phố Hà Nội
2  Quảng Ninh  Thành phố Long Xuyên
3  Quảng Ninh         Huyện Tri Tôn
4  Quảng Ninh          Huyện Gò Dầu
Tổng số tuyến: 9506
